In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import StandardScaler

# Define the model architecture (same as in the training script)
class WeatherPredictionModel(nn.Module):
    def __init__(self, input_size):
        super(WeatherPredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Load the model weights
def load_model(model_path, input_size):
    model = WeatherPredictionModel(input_size)
    model.load_state_dict(torch.load(model_path))
    model.eval()  # Set to evaluation mode
    return model

# Prediction function
def predict_temperature(model, scaler, year, month, day):
    # Create cyclical features for the input date
    day_sin = np.sin(2 * np.pi * day / 31)
    day_cos = np.cos(2 * np.pi * day / 31)
    month_sin = np.sin(2 * np.pi * month / 12)
    month_cos = np.cos(2 * np.pi * month / 12)
    
    # Input features
    input_data = [[year, day_sin, day_cos, month_sin, month_cos]]
    
    # Scale the input features
    input_data_scaled = scaler.transform(input_data)
    input_tensor = torch.tensor(input_data_scaled, dtype=torch.float32)
    
    # Make prediction
    with torch.no_grad():
        prediction = model(input_tensor)
    return prediction.item()

# Example usage in another program
if __name__ == "__main__":
    # Initialize the scaler (use the same scaler used during training)
    scaler = StandardScaler()
    scaler.mean_ = np.array([2019.5, 0, 0, 0, 0])  # Set the same mean values used in training
    scaler.scale_ = np.array([2.87, 0.72, 0.72, 0.50, 0.50])  # Set the same scale values used in training
    
    # Load the model
    model_path = r"C:\Users\THARUN\wheather prediction\best_weather_prediction_model.pt"
    input_size = 5  # Adjust based on your input features (Year, Day_sin, Day_cos, Month_sin, Month_cos)
    model = load_model(model_path, input_size)
    
    # Example prediction for a specific date
    year = int(input("Enter the year:"))
    month = int(input("Enter the month:"))
    day = int(input("Enter the date:"))
    predicted_temp = predict_temperature(model, scaler, year, month, day)
    print(f'Predicted temperature for {year}-{month}-{day}: {predicted_temp:.2f}°C')


C:\Users\THARUN\AppData\Local\Temp\ipykernel_10424\2935990019.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Predicted temperature for 2024-10-20: 23.32°C
